In [ ]:
# %pip install --upgrade pandas seaborn

In [1]:
# Ignore future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# import external packages
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


# Add path with self-created packages and import them
import sys
sys.path.append('./src')
import sciebo

#### Download Dataset

This data has been pre-processed from Steinmetz and al 2019, and is hosted on Sciebo here: https://uni-bonn.sciebo.de/s/wjsBtZzUVjKaB3J.  The code below should download it to the folder `data/steinmetz_all.csv`

In [2]:
sciebo.download_from_sciebo('https://uni-bonn.sciebo.de/s/wjsBtZzUVjKaB3J', 'data/steinmetz_all.csv')

## Analyzing Behavioral Task Performance with Pandas and Seaborn: Psychometric Analysis on Ordered Categorical Data

In the experiment reported in by [Steinmetz et al, 2019 in Nature](https://www.nature.com/articles/s41586-019-1787-x), mice were tasked with turning a wheel to the left or right based on the relative contrast levels of two simultaneously-presented gradient stimuli:

<img src="https://uni-bonn.sciebo.de/index.php/apps/files_sharing/ajax/publicpreview.php?x=2560&y=921&a=true&file=steinmetz_task2.png&t=gulZ14DB3llTenW&scalingup=0" width=300>


##### Analysis Goals

In this notebook, we'll examine the `response_time` and `response_type` of each trial across all sessions, to determine whether the mice successfully performed the task, and whether the difference in contrast levels between the two stimuli affected their performance.  


##### Learning Goals

We'll use this as an opportunity to get to know the [**Seaborn**](https://seaborn.pydata.org/) Python package's core syntax.  Seaborn is a statistical plotting library that takes Pandas Dataframes and turns them into plots (including making errorbars)!  

--- 

**Load Data**: Use the `pd.read_csv()` function to load the `steinmetz_all.csv` file in the `data` folder into a Pandas DataFrame:

In [83]:
df = pd.read_csv('data/steinmetz_all.csv')
df.head()

Note that our CSV file has this column called `"Unnamed: 0"`, which is not really wanted. We can drop it using the `.drop` method of the Pandas DataFrame:

In [84]:
df = df.drop(columns="Unnamed: 0")
df.head()

---

### Performance Analysis:  Average Response Type for Each Stimulus Contrast Level

How did the mice perform in the task, overall?  Let's use seaborn to make a basic statistical analysis comparing different variables against each other.


What makes Seaborn particularly nice is that most of its functions have the same syntax:

```python
sns.typeofplot(       #  Function name (what type of plot do you want to make?
    data=df,          #  Dataframe variable (what data will this plot be made from?)
    x="column1",      #  Column to use for the x axis of the plot.
    y="column2",      #  Column to use for the y axis of the plot.
    hue="column3",    #  Column to use for splitting the data into different colors.
    ...               # ...more columns can be added, to make a richer and more complex plot!
)
```

In this notebook, we'll look at the plots that help compare a continuous variable across different levels of a categorical variable:

| Plotting Function | Description | Example |
| :-- | :-- | :-- |
| **`sns.barplot()`** | A bar plot | `sns.barplot(data=df, x='mouse', y='response_time')` |
| **`sns.pointplot()`** | a plot with errobars at markers for each category level, and a line connecting each point. | `sns.pointplot(dta=df, x='mouse', y='response_time')`   |

**Exercises**: 

On average, how did the subjects respond when presented with various contrast levels on the right and left stimuli? For each exercise, we'll use seaborn to make the requested plot.

#### Bar Plots

**Example**: Make a bar plot with `contrast_right` levels on the x-axis and `response_type` on the y-axis.  What do you observe?

In [46]:
sns.barplot(data=df, x='contrast_right', y='response_type');

Make a bar plot with `contrast_left` levels on the x-axis and `response_type` on the y-axis.  What do you observe?

Let's create the same plot but "transposed": such that we have the bars horizontally instead of vertically. 

Make a bar plot with `response_type` as the x-axis and `contrast_left` levels on the y-axis.  What does seaborn do here? Is this what we want? (note: to fix this, you can add the argument `orient='h'`. By default, seaborn has `orient='v'`)

Let's combine the x-axis and the hue together!  Make a bar plot with `contrast_left` levels on both the x-axis and hue and `response_type` on the y-axis. (Note: If you don't like having the legend anymore, you can additionally set `legend=False`.)

Make a bar plot with `contrast_left` levels on the x-axis and `response_type` as the `hue` (i.e. the color of the bars). No need to specify anything for the y-axis. What do you observe?

How does the response_time look like for different combinations of left_contrast and right_contrast?  Make a bar plot with `contrast_left` levels on the x-axis, `contrast_right` on the hue, and `response_type` on the y-axis.  Waht's nice about this plot, and what's not so nice?

##### Point Plots

**Example**: Make a point plot with `contrast_right` on the x-axis and `response_type` on the y-axis.  What do you observe?

In [52]:
sns.pointplot(data=df, x='contrast_right', y='response_type');

Make a point plot with `contrast_left` on the x-axis and `response_type` on the y-axis.  What do you observe?

Call the `sns.pointplot()` function twice (on two seperate lines), first with `contrast_left` on the x-axis and then with `contrast_right` on the x-axis.  What do you observe?  What's good and bad about this plot?

How does the left_contrast values compare with the right_contrast values?  Make a point plot with `contrast_left` levels on the x-axis, `contrast_right` on the hue, and `response_type` on the y-axis. What do you observe?  (Note: to shift values a bit to keep errorbars from obscuring each other, try setting `dodge=True`)

---

### Interpreting and Setting Error Bar Types: Confidence Interval vs Standard Deviation vs Standard Error

Bar plots and point plots show "point estimates" of a variable; also known as "aggregations" or "descriptive statistics".  For example, the mean and median are point estimates; they take a range of data and estimate its center at a single point.

Error bars are essential because they tell us either the variation in the data underlying that estimate, or alternatively the quality of the estimate (by showing the range of uncertainty underlying it).  Here are the four most common types of errorbars and their purposes:

| Error Bar Type | Unit | Purpose | Implies Normal Distribution? | Seaborn Setting Example | Interpretation of Example |
| :-- | :-- | :-- | :-- | :-- | :-- |
| Standard Deviation | variable's unit | Show Data Variation | Yes | `errorbar=('sd', 1)` | "One Standard Deviation" | 
| Percentile Interval | percent | Show Data Variation | No | `errorbar=('pi', 95)` | "95% Intervals" | 
| Standard Error (of the Mean) | variable's unit | Show Estimate Uncertainty | Yes | `errorbar=('se', 1)` | "One Standard Error" | 
| Confidence Intervals | percent | Show Estimate Uncertainty | No | `errorbar=('ci', 95)` | "95% Confidence Intervals" |


Seaborn by default calculates the mean of the data, and shows its uncertanty using a 95% Confidence Interval.  This is a good default, but it can be changed to other settings.  Let's get a feel for these error bar types!

**Exercises**: 

In each of the following exercises, explore the errorbars by re-making the last plot in the previous section, setting the errobars to different types and levels:

**Example**: Make 95% confidence intervals.

In [56]:
sns.pointplot(data=df, x='contrast_left', hue='contrast_right', y='response_type', dodge=True, errorbar=('ci', 95));

Make 0% confidence intervals.  What do you expect to see?

Make 25% confidence intervals.  What do you expect to see?

Make 99% confidence intervals. What do you expect to see?

Set the error bars to show one standard deviation (`sd`) of the data.  What do you notice about these errobars, compared to the ones made previously?

Set the error bars to show three standard deviations of the data (roughly the full range of data's variation in the dataset, assuming normally-distributed data)

Account for the number of observations by setting the error bars to show one "Standard Error of the Mean" (`se`)

Set the error bars to show two Standard Errors of the Mean, and compare this to the plot showing a 95% Confidence Interval.  What do you notice?

**(Extra)** Set the error bars to show 50% Intervals (`pi`).  What is strange about this plot?  Try setting it to different levels, and see what you get.  What does this indicate about the data? 

---

## Analyzing Response Time and Response Type: Transforming and Filtering Data, and Comparing Estimates

#### Making New Columns in Pandas DataFrames

| Code | Description |
| :-- | :-- | 
| **`df['column1']`** | Gets a column called "column1" from a DataFrame | 
| **`df['newcolumn'] = 3`** | Makes a new column called "newcolumn" and sets every row to 3 |
| **`df['newcolumn'] = df['column1'] - 3`** | Makes a new column called "newcolumn", where every row is "column1" minus 3 |
| **`df['newcolumn'] = df['column1'] + df['column2']`** | Makes a new column called "newcolumn", which is the sum of "column1" and "column2" |
| **`df['newcolumn'] = df['column1'] > df['column2']`** | Makes a new column called "newcolumn", which is True where "column1" is greater than "column2" and False everywhere else |
| **`df['newcolumn'] = df['column1'] != 5`** | Makes a new column called "newcolumn", which is True where "column1" is not equal to 5 |
| **`df['newcolumn'] = df['column1'].abs()`** | Makes a new column called "newcolumn", which is the absolute value of "column1" |
| **`new_df = df.copy()`** | Makes a copy of the dataframe (useful if you want to keep the old one unchanged) |


**Exercises**: 

In the exercises below, let's practice making new columns in our Pandas DataFrame. We will then use the newly created column in the next set of analyses.

**Example**: How salient, overall, were the stimuli?  Make a new column called `"contrast_total"` that is the sum of the two stimuli's contrast levels.

In [65]:
df['contrast_total'] = df['contrast_right'] + df['contrast_left']
df.sample(3)

The mice had to make a decision of whether to move the wheel left or right based on the *difference* in contrast between the left and right stimulus, not on the actual levels of the data.  Make a new column called `contrast_diff` that contains the contrast difference between the left and right stimuli.
  

The mice were not allowed to respond immediately after the stimuli appeared; instead, they had to wait for the "go cue" to appear, which was at a randomized time point after the stimuli.  Let's calculate the a new `response_time_corrected` column, which subtracts the `gocue_time` from the `response_time`.

Instead of calculating the `contrast_diff`, let's calculate the `contrast_diff_absolute`, where the absolute value of the contrast difference is used, so it is always positive.  This more-closely shows the decision that the mice had to make, and allows us to use twice as many values for each point estimation.

It's only useful to calculate the response time when the mice actually made a response!  Make a column called `did_responsd` that is True when the response_type column is not equal to 0.

### Plotting Response Times and Response Types: Comparing Different Point Estimate Functions

##### Estimators 

Seaborn is a statistical plotting tool, so it gives you control over which statistics you want to plot!  To change which statistic (called an **"estimate"** by Seaborn) is used, set the **`estimator`** parameter to what you need.  Some common estimators like mean, median, and std are already built-in, so you can just write them as a string, but Seaborn will accept any statistics function you'd like it to use:

| Estimator | Passing a String | Passing the Function Directly | 
| :-- | :--  | :-- |
| Mean | **`estimator = 'mean'`** | **`estimator = np.mean`**|
| Median | **`estimator = 'median'`** | **`estimator = np.median`**|
| Standard Deviation | **`estimator = 'std'`** | **`estimator = np.std`** |

This has a big effect on the plots.  Let's try it out!

##### Multilevel Bootstrapping Error Bars Units

If you have a nested structure to your study where observations are grouped (for example, in this study it is made up of sessions), then you should set `units=` to the grouping column; this gets you more-accurate estimates and errorbars that account for between-session varinance. 

**Exercises**:

 Let's see how the estimator used changes the information conveyed in a plot.  For each of the two measurements (response type and response time), which estimators do you think are the most valuable for this experiment?

**Example**: For only the trials where the mouse responded, plot the `mean` response type for each stimulus contrast difference.  Account for the differences between sessions by setting `units` to session_id

In [71]:
sns.pointplot(data=df[df['did_respond']], x='contrast_diff', y='response_type', estimator='mean', units='session_id');

 Where did the response type vary the most?  For only the trials where the mouse responded, plot the `standard deviation` (std) of the response types for each stimulus contrast difference.  Account for the differences between sessions by setting `units` to session_id

**Example**: Is there any relationship between how fast the mice responded and the difference in the contrast?

For only the trials where the mouse responded, plot the `mean` corrected response time for each absolute stimulus contrast difference, using color to show the results for each response type.  Account for the differences between sessions by setting `units` to session_id

In [76]:
sns.pointplot(data=df[df['did_respond']], x='contrast_diff_absolute', hue='response_type', y='response_time_corrected', dodge=True, estimator="mean", units='session_id');

For only the trials where the mouse responded, plot the `median` corrected response time for each absolute stimulus contrast difference, using color to show the results for each response type.  Account for the differences between sessions by setting `units` to session_id

For only the trials where the mouse responded, see where there was the most variance in respond time by plotting the `standard deviation` (std) of the corrected response times for each absolute stimulus contrast difference, using color to show the results for each response type.  Account for the differences between sessions by setting `units` to session_id

## (Demo) Wrapping it Up: Exporting a Figure

Below is an example of how to export our two main results into a single figure with two subplots, then save it as a file (say, for a poster presentation).  

In [79]:
plt.style.available 

**Example**: Run the plot below to see it all put togther, along with subplots and explicit titles and labels; feel free to modify the code (especially the `style='ggplot' part`!)

In [80]:
# Figure 1b
import matplotlib.pyplot as plt

with plt.style.context(style='ggplot', after_reset=True):

    plt.figure(figsize=(16, 7))  # changing the figure size is an quick-and-dirty way to change the font size

    # Subplot 1
    plt.subplot(1, 2, 1)

    df['contrast_diff'] = df['contrast_right'] - df['contrast_left']
    mask = df['response_type'] != 0
    sns.pointplot(data=df[mask], x='contrast_diff', y='response_type', color='dimgrey', units='session_id', n_boot=300);
    plt.title('Performance Reaches >80%\nAbove a 50% Difference in Contrast')
    plt.xlabel('Contrast Difference (%)')
    plt.ylabel("Response Type")

    # Subplot 2
    plt.subplot(1, 2, 2)

    mask = df['response_type'] != 0
    df['contrast_diff_absolute'] = df['contrast_diff'].abs()
    sns.pointplot(
        data=df[mask].astype({'response_type': 'category'}), 
        x='contrast_diff_absolute', y='response_time_corrected',
        hue='response_type', 
        units='session_id', 
        dodge=True,
        n_boot=300,
    );
    plt.title('Response Time Decreases as \n Stimulus Contrast Difference Increases', )
    plt.xlabel('Absolute Contrast Difference (%)')
    plt.ylabel("Response Time after Go Cue")
    plt.legend(title="Response Type", frameon=False);

    # Correct the spacing between subplots, to fix some kinds of accidental overlap.
    plt.tight_layout()

    # Save the figure 
    # plt.savefig('performance.svg', dpi=200)  # for editing later in a vector graphics editor (i.e. Inkscape, Adobe Illustrator)
    plt.savefig('performance.png', dpi=200)  
